In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Investigate break-up momentum 
Implement phasespace factor with cleaner cut structure:
$$
\begin{aligned}
\rho_\alpha(s) &= \frac{\sqrt{s-(m_{1,\alpha}-m_{2,\alpha})^2}\sqrt{s-(m_{1,\alpha}+m_{2,\alpha})^2}}{s} \\
q_\alpha(s) &= \frac{\sqrt{s-(m_{1,\alpha}-m_{2,\alpha})^2}\sqrt{s-(m_{1,\alpha}+m_{2,\alpha})^2}}{2\sqrt{s}}
\end{aligned}
$$

In [ ]:
from __future__ import annotations

from typing import TYPE_CHECKING, Any

import sympy as sp

from ampform.sympy import argument, determine_indices, unevaluated

if TYPE_CHECKING:
    from sympy.printing.latex import LatexPrinter

from ampform.dynamics.phasespace import BreakupMomentumSquared, _indices_to_subscript

## Old definition of break-up momentum and phasespace factor

In [ ]:
@unevaluated
class PhaseSpaceFactorOld(sp.Expr):
    r"""Standard phase-space factor, using :func:`BreakupMomentumSquared`.

    See :pdg-review:`2021; Resonances; p.6`, Equation (50.9). We ignore the factor
    :math:`\frac{1}{16\pi}` as done in :cite:`chungPrimerKmatrixFormalism1995`, p.5.
    """

    s: Any
    m1: Any
    m2: Any
    name: str | None = argument(default=None, sympify=False)

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q_squared = BreakupMomentumSquared(s, m1, m2)
        return 2 * sp.sqrt(q_squared) / sp.sqrt(s)

    def _latex_repr_(self, printer: LatexPrinter, *args) -> str:
        s_symbol = self.args[0]
        s_latex = printer._print(s_symbol)
        subscript = _indices_to_subscript(determine_indices(s_symbol))
        name = R"\rho" + subscript if self.name is None else self.name
        return Rf"{name}\left({s_latex}\right)"

## Plot phasespace factor and break-up momentum in the complex plane